![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**0- Connect to your Google Drive folder**

In [ ]:
my_folder = "/content/drive/MyDrive/Semestre 7/AI II/SAS_CORTEX"

from google.colab import drive
drive.mount('/content/drive')

# Change the following code to set your Drive folder
import os
os.chdir(my_folder)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Semestre 7/AI II/SAS_CORTEX


**1- Make sure that your Python version is 3.3 or higher as well as your Java version is 1.8.0_162 or higher**

In [ ]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [ ]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [ ]:
pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 4.2 MB/s 
  Created wheel for saspy: filename=saspy-4.4.1-py3-none-any.whl size=9937304 sha256=1bb2a17744dc1614c33450d1d5bcf5c847880328b619b06313730ff7a9c06a2f
  Stored in directory: /root/.cache/pip/wheels/0a/f6/ab/f7173c8a824856c66f6fd50194bb8fd3ef93fb4f1824be8d0c
Successfully built saspy


**3- Create the configuration file "sascfg_personal.py"**
Please, check that your Home Region is correct, you can check it at [ODA-SAS](https://welcome.oda.sas.com/home)

In [ ]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 1
'iomhost' : ['odaws01-usw2.oda.sas.com','odaws02-usw2.oda.sas.com','odaws03-usw2.oda.sas.com','odaws04-usw2.oda.sas.com'],
#US Home Region 2
#'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
#European Home Region 1
#'iomhost' : ['odaws01-euw1.oda.sas.com','odaws02-euw1.oda.sas.com'],
#Asia Pacific Home Region 1
#'iomhost' : ['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'],
#Asia Pacific Home Region 2
#'iomhost' : ['odaws01-apse1-2.oda.sas.com','odaws02-apse1-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create your .authinfo**

If there is no .authinfo file, you can create this

In [ ]:
#%%writefile .authinfo
#oda user USR password PSW

Copy this file to home

In [ ]:
!cp .authinfo ~/.authinfo

**5- Establish Connection (Need to do this step each time you use SASPy)**

In [ ]:
import saspy
sas_session = saspy.SASsession(cfgfile=os.path.join(
    my_folder,"sascfg_personal.py"))
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 378



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/Semestre 7/AI II/SAS_CORTEX/sascfg_personal.py
WORK Path             = /saswork/SAS_workBCA40000FAED_odaws03-usw2.oda.sas.com/SAS_work0BEF0000FAED_odaws03-usw2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 64237


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [135]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


133                                                        The SAS System                      Friday, December  2, 2022 02:12:00 PM

930        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
930      ! ods graphics on / outputfmt=png;
931        
932        
933            libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
934        
935        
936        
937        ods html5 (id=saspy_internal) close;ods listing;
938        

134                                                        The SAS System                      Friday, December  2, 2022 02:12:00 PM

939        


For local Jupyter

In [136]:
#%%SAS sas_session
#libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [137]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [164]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
559825,2559826.0,GONZALEZ,MARK,0.0,38.0,78700.0,University / College,Suburban,7.0,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,200.0
626415,2626416.0,CADENA,DEREK,0.0,50.0,42700.0,University / College,Suburban,10.0,7.0,...,7.0,9.0,405.0,10.0,250.0,0.0,0.0,0.0,1.0,20.0


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [ ]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)  

data_merge.sample(5)

## Data Partition

In [181]:
df.columns

Index(['ID', 'Woman', 'Age', 'Salary', 'Education', 'City', 'SeniorList',
       'NbActivities', 'Referrals', 'Recency', 'Frequency', 'Seniority',
       'TotalGift', 'MinGift', 'MaxGift', 'GaveLastYear', 'AmtLastYear',
       'Contact', 'GaveThisYear', 'AmtThisYear'],
      dtype='object')

In [182]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
#train, validation = train_test_split(data_merge, test_size=0.5,random_state=5678) # you can change the percentage
x = df[['ID','Woman', 'Age', 'Salary', 'Education', 'City', 'SeniorList','NbActivities', 'Referrals', 'Recency', 'Frequency', 'Seniority',
       'TotalGift', 'MinGift', 'MaxGift', 'GaveLastYear', 'AmtLastYear','Contact']] 
y = df[['GaveThisYear']]

X_train, X_valid, Y_train, Y_valid = train_test_split(x,y, test_size=0.4, random_state=12345) 



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [183]:
from sklearn import linear_model

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [127]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

0.0
0.0
0.0


## Regression Tree Model（Py）

In [128]:
from sklearn.tree import DecisionTreeRegressor

DT_model = DecisionTreeRegressor(max_depth=5, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[1. 1. 1. ... 1. 1. 1.]


In [129]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

0.0
0.0
0.0


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


## Scoring New Data

### Prepare data for scoring

In [130]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [ ]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

prueba = scoring_data_contact

prueba['MaxGift'] = prueba['MaxGift'].mask(prueba['MaxGift'].isna() , prueba['AmtLastYear'])
prueba['MinGift'] = prueba['MinGift'].mask(prueba['MinGift'].isna(), prueba['AmtLastYear'])
prueba['TotalGift'] = prueba['TotalGift'].mask(prueba['TotalGift'].isna(), (prueba['MaxGift'] + prueba['MinGift']))

prueba['Recency'] = prueba['Recency'].mask((prueba['Recency'].isna()) & (prueba['GaveLastYear'] == 1), 1)
prueba['Recency'] = prueba['Recency'].mask((prueba['Recency'].isna()), prueba['Recency'].median())

prueba['Frequency'] = prueba['Frequency'].mask((prueba['Frequency'].isna()) & (prueba['GaveLastYear'] == 1), 1)
prueba['Frequency'] = prueba['Frequency'].mask((prueba['Frequency'].isna()), prueba['Frequency'].median())

prueba['Seniority'] = prueba['Seniority'].mask((prueba['Seniority'].isna()), prueba['Frequency'])

scoring_data_contact = prueba

scoring_data_contact["Education"] = scoring_data_contact["Education"].astype('category')
scoring_data_contact["City"] = scoring_data_contact["City"].astype('category')

scoring_data_contact["Education"] = scoring_data_contact["Education"].cat.codes
scoring_data_contact["City"] = scoring_data_contact["City"].cat.codes


X = scoring_data_contact[['ID','Woman', 'Age', 'Salary', 'SeniorList','NbActivities', 'Referrals', 'Recency', 'Frequency', 'Seniority',
       'TotalGift', 'MinGift', 'MaxGift', 'GaveLastYear', 'AmtLastYear','Contact']] 

regr_predict_contact=regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

,ID,AmtContact
0,2000001.0,1.0
1,2000002.0,1.0
2,2000003.0,1.0
3,2000004.0,1.0
4,2000005.0,1.0


### Predict 'amount given' for members who were not contacted

In [ ]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

prueba = scoring_data_nocontact

prueba['MaxGift'] = prueba['MaxGift'].mask(prueba['MaxGift'].isna() , prueba['AmtLastYear'])
prueba['MinGift'] = prueba['MinGift'].mask(prueba['MinGift'].isna(), prueba['AmtLastYear'])
prueba['TotalGift'] = prueba['TotalGift'].mask(prueba['TotalGift'].isna(), (prueba['MaxGift'] + prueba['MinGift']))

prueba['Recency'] = prueba['Recency'].mask((prueba['Recency'].isna()) & (prueba['GaveLastYear'] == 1), 1)
prueba['Recency'] = prueba['Recency'].mask((prueba['Recency'].isna()), prueba['Recency'].median())

prueba['Frequency'] = prueba['Frequency'].mask((prueba['Frequency'].isna()) & (prueba['GaveLastYear'] == 1), 1)
prueba['Frequency'] = prueba['Frequency'].mask((prueba['Frequency'].isna()), prueba['Frequency'].median())

prueba['Seniority'] = prueba['Seniority'].mask((prueba['Seniority'].isna()), prueba['Frequency'])

scoring_data_nocontact = prueba

scoring_data_nocontact["Education"] = scoring_data_nocontact["Education"].astype('category')
scoring_data_nocontact["City"] = scoring_data_nocontact["City"].astype('category')

scoring_data_nocontact["Education"] = scoring_data_nocontact["Education"].cat.codes
scoring_data_nocontact["City"] = scoring_data_nocontact["City"].cat.codes

X = scoring_data_nocontact[['ID','Woman', 'Age', 'Salary', 'SeniorList','NbActivities', 'Referrals', 'Recency', 'Frequency', 'Seniority',
       'TotalGift', 'MinGift', 'MaxGift', 'GaveLastYear', 'AmtLastYear','Contact']] 
 

regr_predict_nocontact=regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,AmtNoContact
0,2000001.0,1.0
1,2000002.0,1.0
2,2000003.0,1.0
3,2000004.0,1.0
4,2000005.0,1.0


In [ ]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,1.0,1.0
1,2000002.0,1.0,1.0
2,2000003.0,1.0,1.0


## Exporting Results to a CSV File

In [ ]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.

In [ ]:
!head Round2_Output_amt.csv

ID,AmtContact,AmtNoContact
2000001.0,70.24721894574495,72.01961379511127
2000002.0,64.63982687657455,66.41222172594088
2000003.0,65.06328491430301,66.83567976366933
2000004.0,40.01847884806918,41.7908736974355
2000005.0,64.28022930224044,66.05262415160676
2000006.0,33.09699705382326,34.86939190318959
2000007.0,49.54199001404537,51.31438486341169
2000008.0,60.38208123006558,62.15447607943191
2000009.0,25.91536090148847,27.687755750854798
